In [1]:
import pandas as pd
import numpy as np
import argparse
import torch
import random

In [3]:
data_dir = '/content/drive/MyDrive/Neural-CF/Yelp-Dataset/subset_250k-user_cnt/subset_review.json'

In [4]:
# Load Data
print('Loading Data....')
user_item_interactions = df = pd.read_json(data_dir, lines=True)
df = pd.DataFrame(user_item_interactions)
df = df.groupby(['user_id', 'business_id']).agg({'stars': 'mean'}).reset_index()

user_id = df[['user_id']].drop_duplicates().reindex()
user_id['userId'] = np.arange(len(user_id))
ml1m_rating = pd.merge(df, user_id, on=['user_id'], how='left')

item_id = df[['business_id']].drop_duplicates()
item_id['itemId'] = np.arange(len(item_id))
yelp_rating = pd.merge(ml1m_rating, item_id, on=['business_id'], how='left')
yelp_rating = yelp_rating[['userId', 'itemId', 'stars']]
yelp_rating.rename(columns={'stars': 'rating'}, inplace=True)

Loading Data....


In [6]:
yelp_rating.to_csv('/content/drive/MyDrive/Neural-CF/Yelp-Dataset/yelp_rating', index=False)

In [2]:
ratings = pd.read_csv('/content/drive/MyDrive/Neural-CF/Yelp-Dataset/yelp_rating')
item_pool = set(ratings['itemId'].unique())

In [4]:
def _sample_negative(ratings):
    """return all negative items & 100 sampled negative items"""
    print('Generating Negative Items...')

    interact_status = ratings.groupby('userId')['itemId'].apply(set).reset_index().rename(
        columns={'itemId': 'interacted_items'})
    print(interact_status)

    item_pool = set(ratings['itemId'].unique())

    interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: item_pool - x)
    print('-'*50)
    print(interact_status)

    interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 20)) # min(99, len(x))
    print('-'*50)
    print(interact_status)

    # negative_samples_list = []
    # batch_size = 1000
    # item_pool = set(ratings['itemId'].unique())
    # for start in range(0, len(interact_status), batch_size):
    #     batch_interact_status = interact_status[start:start+batch_size].copy()
    #     batch_interact_status['negative_items'] = batch_interact_status['interacted_items'].apply(
    #         lambda x: item_pool - x)
    #     batch_interact_status['negative_samples'] = batch_interact_status['negative_items'].apply(
    #         lambda x: random.sample(x, min(20, len(x))))  # Adjust the sample size as needed
    #     negative_samples_list.append(batch_interact_status[['userId', 'negative_items', 'negative_samples']])
    # result_df = pd.concat(negative_samples_list, ignore_index=True)
    # print('-'*50)
    # print(result_df)

    return interact_status[['userId', 'negative_items', 'negative_samples']]
    # return result_df

In [ ]:
_sample_negative(ratings)

Generating Negative Items...
        userId                                   interacted_items
0            0  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
1            1       {32, 33, 34, 35, 25, 26, 27, 28, 29, 30, 31}
2            2                               {36, 37, 38, 39, 40}
3            3                           {41, 42, 43, 44, 45, 46}
4            4                        {2, 47, 48, 49, 50, 51, 52}
...        ...                                                ...
248486  248486  {19873, 264, 3466, 19051, 15245, 8847, 13426, ...
248487  248487  {10240, 3720, 15499, 1422, 3728, 3741, 30077, ...
248488  248488  {1089, 60705, 48227, 2659, 42429, 6693, 4423, ...
248489  248489  {127648, 87939, 48491, 79757, 114620, 20894, 1...
248490  248490  {8385, 22914, 14851, 33061, 90182, 9021, 11812...

[248491 rows x 2 columns]
